# Greeks calculator for European Options

This notebook aims to be a simple tool to compute the greeks of european options through the analytical method. The code is based on that developed in C++ in the website [quantstart.com](quantstart.com), though adapted to Python language.

**Disclaimer: I am not the owner of the piece of code shown below. For more information, refer to the website mentioned before.**

When trading with derivative products, tackling risk management over the open positions is an important part of the job. Hedging requirements may vary depending on the risk amount the investor wishes to assume. The so-called *Greeks* come in handy at this task, as they measure the sensitivity of the price of a derivative upon changes in the parameters affecting the underlying, such as the price, volatility and time to maturity. In particular, the existence of an analytical solution of the Black-Scholes equation for European options makes possible to derive the Greeks of these options in an analytical manner too. However, when there is no analytical solution, we have to draw upon other methods such as MonteCarlo simulation to compute these values.

## Analytical method

- **Delta ($\Delta$)**: rate of change of the option premium with respect to the spot price of the underlying: $\frac{\partial{C}}{\partial{S}}$
- **Gamma ($\Gamma$)**: rate of change of the Delta with respect to the spot prince of the underlying: $\frac{\partial{}}{\partial{S}}\left(\frac{\partial{C}}{\partial{S}}\right)$
- **Vega**: rate of change of the option premium with respect to the volatility of the underlying: $\frac{\partial{C}}{\partial{\sigma}}$
- **Theta ($\Theta$)**: negative derivative of the option premium with respect to the time to maturity: $\frac{\partial{C}}{\partial{t}}$
- **Rho (P)**: derivative of the option premium with respect to the risk-free rate: $\frac{\partial{C}}{\partial{r}}$

In [1]:
from math import exp, sqrt, log, pi

In [2]:
def normpdf(x):
    return (1.0/sqrt(2*pi)*exp(-0.5*x*x))

In [3]:
def normcdf(x):
    k = 1.0/(1.0 + 0.2316419*x)
    ksum = k*(0.319381530 + k*(-0.356563782 + k*(1.781477937 + k*(-1.821255978 + 1.330274429*k))))
    
    if x >= 0.0:
        return (1.0 - (1.0/sqrt(2*pi))*exp(-0.5*x*x) * ksum)
    elif x < 0.0: 
        return 1.0 - normcdf(-x)

In [4]:
def di(i, Expiry, Strike, Spot, Vol, r):
    return (log(Spot/Strike) + (r + ((-1)**(i-1)*0.5*Vol*Vol)*Expiry)/(Vol*sqrt(Expiry)))

In [5]:
def callPrice(Expiry, Strike, Spot, Vol, r):
    return (Spot * normcdf(di(1, Expiry, Strike, Spot, Vol, r))-Strike*exp(-r*Expiry) * normcdf(di(2, Expiry, Strike, Spot, Vol, r)))

def callDelta(Expiry, Strike, Spot, Vol, r):
    return (normcdf(di(1, Expiry, Strike, Spot, Vol, r)))

def callGamma(Expiry, Strike, Spot, Vol, r):
    return (normpdf(di(1, Expiry, Strike, Spot, Vol, r))/(Spot*Vol*sqrt(Expiry)))

def callVega(Expiry, Strike, Spot, Vol, r):
    return Spot*normpdf(di(1, Expiry, Strike, Spot, Vol, r))*sqrt(Expiry)

def callTheta(Expiry, Strike, Spot, Vol, r):
    return -(Spot*normpdf(di(1, Expiry, Strike, Spot, Vol, r))*Vol)/(2*sqrt(Expiry)) - r*Strike*exp(-r*Expiry)*normcdf(di(2, Expiry, Strike, Spot, Vol, r))

def callRho(Expiry, Strike, Spot, Vol, r):
    return Strike*Expiry*exp(-r*Expiry)*normcdf(di(2, Expiry, Strike, Spot, Vol, r))

In [6]:
def putPrice(Expiry, Strike, Spot, Vol, r):
    return (-Spot*normcdf(-di(1, Expiry, Strike, Spot, Vol, r))+Spot*exp(-r*Expiry)*normcdf(-di(2, Expiry, Strike, Spot, Vol, r)))

def putDelta(Expiry, Strike, Spot, Vol, r):
    return (normcdf(di(1, Expiry, Strike, Spot, Vol, r)) - 1)

def putGamma(Expiry, Strike, Spot, Vol, r):
    return (normpdf(di(1, Expiry, Strike, Spot, Vol, r))/(Spot*Vol*sqrt(Expiry)))

def putVega(Expiry, Strike, Spot, Vol, r):
    return Spot*normpdf(di(1, Expiry, Strike, Spot, Vol, r))*sqrt(Expiry)

def putTheta(Expiry, Strike, Spot, Vol, r):
    return (-(Spot*normpdf(di(1, Expiry, Strike, Spot, Vol, r))*Vol)/(2*sqrt(Expiry)) + r*Strike*exp(-r*Expiry)*normcdf(-di(2, Expiry, Strike, Spot, Vol, r)))

def putRho(Expiry, Strike, Spot, Vol, r):
    return -Expiry*Strike*exp(-r*Expiry)*normcdf(-di(2, Expiry, Strike, Spot, Vol, r))

In [7]:
Expiry = 1
Strike = 100
Spot = 100
Volatility = 0.2
r = 0.05

callPrice = callPrice(Expiry, Strike, Spot, Volatility, r)
callDelta = callDelta(Expiry, Strike, Spot, Volatility, r)
callGamma = callGamma(Expiry, Strike, Spot, Volatility, r)
callVega = callVega(Expiry, Strike, Spot, Volatility, r)
callTheta = callTheta(Expiry, Strike, Spot, Volatility, r)
callRho = callRho(Expiry, Strike, Spot, Volatility, r)

print(f'Call Price: {round(callPrice, 5)}')
print(f'Call Delta: {round(callDelta, 5)}')
print(f'Call Gamma: {round(callGamma, 5)}')
print(f'Call Vega: {round(callVega, 5)}')
print(f'Call Theta: {round(callTheta, 5)}')
print(f'Call Rho: {round(callRho, 5)}')

Call Price: 10.45058
Call Delta: 0.63683
Call Gamma: 0.01876
Call Vega: 37.52403
Call Theta: -6.41403
Call Rho: 53.23248


In [8]:
putPrice = putPrice(Expiry, Strike, Spot, Volatility, r)
putDelta = putDelta(Expiry, Strike, Spot, Volatility, r)
putGamma = putGamma(Expiry, Strike, Spot, Volatility, r)
putVega = putVega(Expiry, Strike, Spot, Volatility, r)
putTheta = putTheta(Expiry, Strike, Spot, Volatility, r)
putRho = putRho(Expiry, Strike, Spot, Volatility, r)

print(f'Put price: {round(putPrice, 5)}')
print(f'Put Delta: {round(putDelta, 5)}')
print(f'Put Gamma: {round(putGamma, 5)}')
print(f'Put Vega: {round(putVega, 5)}')
print(f'Put Theta: {round(putTheta, 5)}')
print(f'Put Rho: {round(putRho, 5)}')

Put price: 5.57352
Put Delta: -0.36317
Put Gamma: 0.01876
Put Vega: 37.52403
Put Theta: -1.65788
Put Rho: -41.89046


## MonteCarlo + Finite Difference Method

In [9]:
from random import random

In [10]:
def GetOneGaussianByBoxMuller():
    sizeSquared = 1
    while sizeSquared >= 1.0:
        x = 2.0*random() - 1
        y = 2.0*random() - 1
        sizeSquared = x*x + y*y
    
    result = x*sqrt(-2*log(sizeSquared)/sizeSquared)
    return result

In [11]:
def SimpleMonteCarloSpot(Expiry, Strike, Spot, Vol, r, deltaSpot, priceSp, priceS, priceSm, NumberOfPaths, OptionType):
    variance = Vol*Vol*Expiry
    rootVariance = sqrt(variance)
    itoCorrection = -0.5*variance
    
    SpAdjust = (Spot+deltaSpot)*exp(r*Expiry + itoCorrection)
    SAdjust = Spot*exp(r*Expiry + itoCorrection)
    SmAdjust = (Spot-deltaSpot)*exp(r*Expiry + itoCorrection)

    SpCur = 0
    SCur = 0
    SmCur = 0

    payoffSumP = 0
    payoffSum = 0
    payoffSumM = 0
    
    if OptionType == "call":
        for i in range(NumberOfPaths):
            thisGaussian = GetOneGaussianByBoxMuller()
            SpCur = SpAdjust * exp(rootVariance*thisGaussian)
            SCur = SAdjust * exp(rootVariance*thisGaussian)
            SmCur = SmAdjust * exp(rootVariance*thisGaussian)

            payoffSumP += max(SpCur - Strike, 0)
            payoffSum += max(SCur - Strike, 0)
            payoffSumM += max(SmCur - Strike, 0)

    elif OptionType == "put":
        for i in range(NumberOfPaths):
            thisGaussian = GetOneGaussianByBoxMuller()
            SpCur = SpAdjust * exp(rootVariance*thisGaussian)
            SCur = SAdjust * exp(rootVariance*thisGaussian)
            SmCur = SmAdjust * exp(rootVariance*thisGaussian)

            payoffSumP += max(Strike - SpCur, 0)
            payoffSum += max(Strike - SCur, 0)
            payoffSumM += max(Strike - SmCur, 0)
            
    priceSp = (payoffSumP/NumberOfPaths) * exp(-r*Expiry)
    priceS = (payoffSum/NumberOfPaths) * exp(-r*Expiry)
    priceSm = (payoffSumM/NumberOfPaths) * exp(-r*Expiry)
        
    return priceSp, priceS, priceSm

In [12]:
def SimpleMonteCarloVol(Expiry, Strike, Spot, Vol, r, deltaVol, priceVolp, priceVol, NumberOfPaths, OptionType):
    variancePAdjust = (Vol+deltaVol)*(Vol+deltaVol)*Expiry
    variance = Vol*Vol*Expiry
    rootVarianceP = sqrt(variancePAdjust)
    rootVariance = sqrt(variance)
    itoCorrectionP = -0.5*variancePAdjust
    itoCorrection = -0.5*variance
    
    thisSpotP = Spot*exp(r*Expiry + itoCorrectionP)
    thisSpot = Spot*exp(r*Expiry + itoCorrection)

    SpCur = 0
    SCur = 0

    payoffSumP = 0
    payoffSum = 0
    
    if OptionType == "call":
        for i in range(NumberOfPaths):
            thisGaussian = GetOneGaussianByBoxMuller()
            SpCur = thisSpotP * exp(rootVarianceP*thisGaussian)
            SCur = thisSpot * exp(rootVariance*thisGaussian)

            payoffSumP += max(SpCur - Strike, 0)
            payoffSum += max(SCur - Strike, 0)

    elif OptionType == "put":
        for i in range(NumberOfPaths):
            thisGaussian = GetOneGaussianByBoxMuller()
            SpCur = thisSpotP * exp(rootVarianceP*thisGaussian)
            SCur = thisSpot * exp(rootVariance*thisGaussian)

            payoffSumP += max(Strike - SpCur, 0)
            payoffSum += max(Strike - SCur, 0)
            
    priceVolp = (payoffSumP/NumberOfPaths) * exp(-r*Expiry)
    priceVol = (payoffSum/NumberOfPaths) * exp(-r*Expiry)
        
    return priceVolp, priceVol

In [13]:
def SimpleMonteCarloTime(Expiry, Strike, Spot, Vol, r, deltaTime, priceTimep, priceTime, NumberOfPaths, OptionType):
    variancePAdjust = Vol*Vol*(Expiry+deltaTime)
    variance = Vol*Vol*Expiry
    rootVarianceP = sqrt(variancePAdjust)
    rootVariance = sqrt(variance)
    itoCorrectionP = -0.5*variancePAdjust
    itoCorrection = -0.5*variance
    
    thisSpotP = Spot*exp(r*(Expiry+deltaTime) + itoCorrectionP)
    thisSpot = Spot*exp(r*Expiry + itoCorrection)

    SpCur = 0
    SCur = 0

    payoffSumP = 0
    payoffSum = 0
    
    if OptionType == "call":
        for i in range(NumberOfPaths):
            thisGaussian = GetOneGaussianByBoxMuller()
            SpCur = thisSpotP * exp(rootVarianceP*thisGaussian)
            SCur = thisSpot * exp(rootVariance*thisGaussian)

            payoffSumP += max(SpCur - Strike, 0)
            payoffSum += max(SCur - Strike, 0)

    elif OptionType == "put":
        for i in range(NumberOfPaths):
            thisGaussian = GetOneGaussianByBoxMuller()
            SpCur = thisSpotP * exp(rootVarianceP*thisGaussian)
            SCur = thisSpot * exp(rootVariance*thisGaussian)

            payoffSumP += max(Strike - SpCur, 0)
            payoffSum += max(Strike - SCur, 0)
            
    priceTimep = (payoffSumP/NumberOfPaths) * exp(-r*(Expiry+deltaTime))
    priceTime = (payoffSum/NumberOfPaths) * exp(-r*Expiry)
        
    return priceTimep, priceTime

In [14]:
def SimpleMonteCarloRate(Expiry, Strike, Spot, Vol, r, deltaRate, priceRatep, priceRate, NumberOfPaths, OptionType):
    variance = Vol*Vol*Expiry
    rootVariance = sqrt(variance)
    itoCorrection = -0.5*variance
    
    thisSpotP = Spot*exp((r+deltaRate)*Expiry + itoCorrection)
    thisSpot = Spot*exp(r*Expiry + itoCorrection)

    SpCur = 0
    SCur = 0

    payoffSumP = 0
    payoffSum = 0
    
    if OptionType == "call":
        for i in range(NumberOfPaths):
            thisGaussian = GetOneGaussianByBoxMuller()
            SpCur = thisSpotP * exp(rootVariance*thisGaussian)
            SCur = thisSpot * exp(rootVariance*thisGaussian)

            payoffSumP += max(SpCur - Strike, 0)
            payoffSum += max(SCur - Strike, 0)

    elif OptionType == "put":
        for i in range(NumberOfPaths):
            thisGaussian = GetOneGaussianByBoxMuller()
            SpCur = thisSpotP * exp(rootVariance*thisGaussian)
            SCur = thisSpot * exp(rootVariance*thisGaussian)

            payoffSumP += max(Strike - SpCur, 0)
            payoffSum += max(Strike - SCur, 0)
            
    priceRatep = (payoffSumP/NumberOfPaths) * exp(-(r+deltaRate)*Expiry)
    priceRate = (payoffSum/NumberOfPaths) * exp(-r*Expiry)
        
    return priceRatep, priceRate

In [15]:
def DeltaMonteCarlo(Expiry, Strike, Spot, Vol, r, deltaSpot, NumberOfPaths, OptionType):
    priceSp = 0
    priceS = 0
    priceSm = 0
    
    price = SimpleMonteCarloSpot(Expiry, Strike, Spot, Vol, r, deltaSpot, priceSp, priceS, priceSm, NumberOfPaths, OptionType)
    return ((price[0] - price[1])/deltaSpot)

In [16]:
def GammaMonteCarlo(Expiry, Strike, Spot, Vol, r, deltaSpot, NumberOfPaths, OptionType):
    priceSp = 0
    priceS = 0
    priceSm = 0
    
    price = SimpleMonteCarloSpot(Expiry, Strike, Spot, Vol, r, deltaSpot, priceSp, priceS, priceSm, NumberOfPaths, OptionType)
    return (price[0] - 2*price[1] + price[2])/(deltaSpot*deltaSpot)

In [17]:
def VegaMonteCarlo(Expiry, Strike, Spot, Vol, r, deltaVol, NumberOfPaths, OptionType):
    priceVolp = 0
    priceVol = 0
    
    price = SimpleMonteCarloVol(Expiry, Strike, Spot, Vol, r, deltaVol, priceVolp, priceVol, NumberOfPaths, OptionType)
    return ((price[0] - price[1])/deltaVol)

In [18]:
def ThetaMonteCarlo(Expiry, Strike, Spot, Vol, r, deltaTime, NumberOfPaths, OptionType):
    priceTimep = 0
    priceTime = 0
    
    price = SimpleMonteCarloTime(Expiry, Strike, Spot, Vol, r, deltaTime, priceTimep, priceTime, NumberOfPaths, OptionType)
    return (-(price[0] - price[1])/deltaTime)

In [19]:
def RhoMonteCarlo(Expiry, Strike, Spot, Vol, r, deltaRate, NumberOfPaths, OptionType):
    priceRatep = 0
    priceRate = 0
    
    price = SimpleMonteCarloRate(Expiry, Strike, Spot, Vol, r, deltaRate, priceRatep, priceRate, NumberOfPaths, OptionType)
    return ((price[0] - price[1])/deltaRate)

In [20]:
Expiry = 1
Strike = 100
Spot = 100
Volatility = 0.2
r = 0.05
deltaSpot = 0.001
deltaVol = 0.001
deltaTime = 0.001
deltaRate = 0.001
NumberOfPaths = 10000000
CallOption = "call"
PutOption = "put"

In [21]:
callDelta = DeltaMonteCarlo(Expiry, Strike, Spot, Volatility, r, deltaSpot, NumberOfPaths, CallOption)
callGamma = GammaMonteCarlo(Expiry, Strike, Spot, Volatility, r, deltaSpot, NumberOfPaths, CallOption)
callVega = VegaMonteCarlo(Expiry, Strike, Spot, Volatility, r, deltaVol, NumberOfPaths, CallOption)
callTheta = ThetaMonteCarlo(Expiry, Strike, Spot, Volatility, r, deltaTime, NumberOfPaths, CallOption)
callRho = RhoMonteCarlo(Expiry, Strike, Spot, Volatility, r, deltaRate, NumberOfPaths, CallOption)

putDelta = DeltaMonteCarlo(Expiry, Strike, Spot, Volatility, r, deltaSpot, NumberOfPaths, PutOption)
putGamma = GammaMonteCarlo(Expiry, Strike, Spot, Volatility, r, deltaSpot, NumberOfPaths, PutOption)
putVega = VegaMonteCarlo(Expiry, Strike, Spot, Volatility, r, deltaVol, NumberOfPaths, PutOption)
putTheta = ThetaMonteCarlo(Expiry, Strike, Spot, Volatility, r, deltaTime, NumberOfPaths, PutOption)
putRho = RhoMonteCarlo(Expiry, Strike, Spot, Volatility, r, deltaRate, NumberOfPaths, PutOption)

print(f'Call Delta: {round(callDelta, 6)}')
print(f'Call Gamma: {round(callGamma, 6)}')
print(f'Call Vega: {round(callVega, 6)}')
print(f'Call Theta: {round(callTheta, 6)}')
print(f'Call Rho: {round(callRho, 6)}')

print(f'Put Delta: {round(putDelta, 6)}')
print(f'Put Gamma: {round(putGamma, 6)}')
print(f'Put Vega: {round(putVega, 6)}')
print(f'Put Theta: {round(putTheta, 6)}')
print(f'Put Rho: {round(putRho, 6)}')

Call Delta: 0.636945
Call Gamma: 0.018367
Call Vega: 37.605031
Call Theta: -6.414979
Call Rho: 53.308422
Put Delta: -0.363395
Put Gamma: 0.020236
Put Vega: 37.503961
Put Theta: -1.656607
Put Rho: -41.761428
